# Semantic Search Engine


First, we start off by importing all the required packages and loading the environment variables.


In [22]:
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import chain
from qdrant_client.models import Distance, VectorParams

load_dotenv(
    "/Users/aryankhurana/Developer/GenAI-with-Langchain/projects/02-semantic-search-engine/.env"
)

True

The `PyPDFLoader` class is used to load the PDF document and extract the text from it in a format that is readable by LangChain. Each document that is loaded is typically a page which has the content of the page and some metadata about it.


In [23]:
file_path = "/Users/aryankhurana/Developer/GenAI-with-Langchain/projects/02-semantic-search-engine/nke-10k-2023.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()
print(len(docs))

107


In [24]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
F

{'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': '/Users/aryankhurana/Developer/GenAI-with-Langchain/projects/02-semantic-search-engine/nke-10k-2023.pdf', 'total_pages': 107, 'page': 0, 'page_label': '1'}


We now take the documents that we have and further divide them into chunks based on the parameters that we have configured. This is important because large documents need to be broken down into smaller pieces that fit within context windows for embedding models or LLMs.


In [25]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)

all_splits = text_splitter.split_documents(docs)

len(all_splits)

516

An embedding model is responsible for creating text into high-dimensional vector representations


In [26]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

Next, we are converting the text content of the first two chunks into numerical vector embeddings. `3072` which is the length of the vector, is the dimensionality.

When we talk about "dimensions" in the context of embeddings, each number in the array represents a separate dimension in a mathematical vector space. So this is a vector in a 3072-dimensional vector space, but it's stored as a simple 1D array with 3072 elements.

This high-dimensional representation allows the embedding model to capture complex semantic relationships and meanings in the text, far beyond what a 2D or 3D representation could capture. Each dimension potentially represents some aspect or feature of the text's meaning that the model has learned during training.


In [27]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 3072

[0.00927501916885376, -0.015898244455456734, 0.0002826128911692649, 0.006466870661824942, 0.020663965493440628, -0.039251528680324554, -0.007505072746425867, 0.041077762842178345, -0.007980394177138805, 0.05989047884941101]


Now we will create a Qdrant client in Memory that will exist only for the duration of the program's execution. We give the qdrant database access to the embeddings model that we created earlier which it will use to convert text to vectors when adding documents. Now, we will create a collection called test and then initialize the client.


In [28]:
client = QdrantClient(":memory:")
client.create_collection(
    collection_name="test",
    vectors_config=VectorParams(
        size=3072,  # Matches the dimension of your embeddings
        distance=Distance.COSINE,  # Or another distance metric like Distance.EUCLID or Distance.DOT
    ),
)
vector_store = QdrantVectorStore(
    client=client,
    collection_name="test",
    embedding=embeddings,
)

Next, we will add all our documents to the vector database.


In [29]:
ids = vector_store.add_documents(documents=all_splits)

After this is done, we can now use natural language to carry out semantic search.


In [36]:
results = vector_store.similarity_search("How does Nike account for income tax?")

print(results[0])

page_content='On August 16, 2022, the U.S. government enacted the Inflation Reduction Act of 2022 that includes, among other provisions, changes to the U.S. corporate income tax
system, including a fifteen percent minimum tax based on "adjusted financial statement income," which is effective for NIKE beginning June 1, 2023. Based on our current
analysis of the provisions, we do not expect these tax law changes to have a material impact on our financial statements; however, we will continue to evaluate their
impact as further information becomes available.
2023 FORM 10-K 35' metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'sourc

We can also do this asynchronously


In [31]:
results = await vector_store.asimilarity_search("When was Nike incorporated?")

print(results[0])

page_content='Table of Contents
PART I
ITEM 1. BUSINESS
GENERAL
NIKE, Inc. was incorporated in 1967 under the laws of the State of Oregon. As used in this Annual Report on Form 10-K (this "Annual Report"), the terms "we," "us," "our,"
"NIKE" and the "Company" refer to NIKE, Inc. and its predecessors, subsidiaries and affiliates, collectively, unless the context indicates otherwise.
Our principal business activity is the design, development and worldwide marketing and selling of athletic footwear, apparel, equipment, accessories and services. NIKE is
the largest seller of athletic footwear and apparel in the world. We sell our products through NIKE Direct operations, which are comprised of both NIKE-owned retail stores
and sales through our digital platforms (also referred to as "NIKE Brand Digital"), to retail accounts and to a mix of independent distributors, licensees and sales' metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'cr

This method also returns a similarity score for each result.


In [32]:
results = vector_store.similarity_search_with_score("What was Nike's revenue in 2023?")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.6886947937194412

page_content='Table of Contents
FISCAL 2023 NIKE BRAND REVENUE HIGHLIGHTSThe following tables present NIKE Brand revenues disaggregated by reportable operating segment, distribution channel and major product line:
FISCAL 2023 COMPARED TO FISCAL 2022
• NIKE, Inc. Revenues were $51.2 billion in fiscal 2023, which increased 10% and 16% compared to fiscal 2022 on a reported and currency-neutral basis, respectively.
The increase was due to higher revenues in North America, Europe, Middle East & Africa ("EMEA"), APLA and Greater China, which contributed approximately 7, 6,
2 and 1 percentage points to NIKE, Inc. Revenues, respectively.
• NIKE Brand revenues, which represented over 90% of NIKE, Inc. Revenues, increased 10% and 16% on a reported and currency-neutral basis, respectively. This
increase was primarily due to higher revenues in Men's, the Jordan Brand, Women's and Kids' which grew 17%, 35%,11% and 10%, respectively, on a wholesale
equivalent basis.' metad

Instead of letting the database handle the embedding process, you can do it yourself as well and then search using the vector


In [37]:
embedding = embeddings.embed_query("How does Nike account for income tax?")

In [38]:
results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

page_content='On August 16, 2022, the U.S. government enacted the Inflation Reduction Act of 2022 that includes, among other provisions, changes to the U.S. corporate income tax
system, including a fifteen percent minimum tax based on "adjusted financial statement income," which is effective for NIKE beginning June 1, 2023. Based on our current
analysis of the provisions, we do not expect these tax law changes to have a material impact on our financial statements; however, we will continue to evaluate their
impact as further information becomes available.
2023 FORM 10-K 35' metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'sourc

LangChain VectorStore objects do not subclass Runnable. LangChain Retrievers are Runnables, so they implement a standard set of methods (e.g., synchronous and asynchronous invoke and batch operations). Although we can construct retrievers from vector stores, retrievers can interface with non-vector store sources of data, as well (such as external APIs).


In [34]:
@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)


retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': '/Users/aryankhurana/Developer/GenAI-with-Langchain/projects/02-semantic-search-engine/nke-10k-2023.pdf', 'total_pages': 107, 'page': 26, 'page_label': '27', 'start_index': 804, '_id': '0159f9086ffd445a95cf721beef6b38e', '_collection_name': 'test'}, page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our\nwholesale, NIKE Direct and merchandising strategies in the region, among other functions.\nIn the United States, NIKE has eight significa

Vectorstores implement an as_retriever method that will generate a Retriever, specifically a VectorStoreRetriever. These retrievers include specific search_type and search_kwargs attributes that identify what methods of the underlying vector store to call, and how to parameterize them. For instance, we can replicate the above with the following:


In [35]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': '/Users/aryankhurana/Developer/GenAI-with-Langchain/projects/02-semantic-search-engine/nke-10k-2023.pdf', 'total_pages': 107, 'page': 26, 'page_label': '27', 'start_index': 804, '_id': '0159f9086ffd445a95cf721beef6b38e', '_collection_name': 'test'}, page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our\nwholesale, NIKE Direct and merchandising strategies in the region, among other functions.\nIn the United States, NIKE has eight significa